In [3]:
import pandas as pd
import requests
import json
import os

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")



In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
stock_list = pd.read_csv("data/equities.csv")
GICS = pd.read_excel("data/GICS.xlsx")

In [6]:
stock_list['symbol'].fillna("NaN", inplace=True)

In [7]:
GICS[['sub_industry_code', 'sub-industry']]

,sub_industry_code,sub-industry
0,10101010,Oil & Gas Drilling
1,10101020,Oil & Gas Equipment & Services
2,10102010,Integrated Oil & Gas
3,10102020,Oil & Gas Exploration & Production
4,10102030,Oil & Gas Refining & Marketing
...,...,...
158,60108050,Data Center REITs
159,60201010,Diversified Real Estate Activities
160,60201020,Real Estate Operating Companies
161,60201030,Real Estate Development


In [8]:
stock_list[['symbol', 'exchange', 'market']].head()

,symbol,exchange,market
0,000002.SZ,SHZ,Shenzhen Stock Exchange
1,000004.SZ,SHZ,Shenzhen Stock Exchange
2,000005.SZ,SHZ,Shenzhen Stock Exchange
3,000006.SZ,SHZ,Shenzhen Stock Exchange
4,000007.SZ,SHZ,Shenzhen Stock Exchange


In [9]:
# Split the original DataFrame by exchange
grouped = stock_list.groupby('exchange')

# Columns to keep for each exchange
columns_to_keep = ['symbol', 'name', 'sector', 'industry_group', 'industry', 'exchange', 'market', 'country', 'website', 'market_cap']

# Create array to store DataFrames
exchange_dict = {}

# Iterate through each group and create new DataFrames
for exchange_code, exchange_group in grouped:
    # Create the new DataFrame using the exchange code
    formatted_exchange_code = exchange_code.lower()
    
    # Select only the specified columns
    selected_columns = columns_to_keep  # Include 'exchange' column for reference
    exchange_group = exchange_group[selected_columns]
    
    # Create the new DataFrame using the formatted code
    globals()[formatted_exchange_code] = exchange_group.copy()
    
    # Add the DataFrame to the exchange_array
    exchange_dict[formatted_exchange_code] = exchange_group.copy()
# Now you will have separate DataFrames for each exchange, containing only the specified columns

In [10]:
exchange_dict['asx']['symbol'] = exchange_dict['asx']['symbol'].apply(lambda x: x.split('.')[0])
asx_codes = exchange_dict['asx'][['symbol']]

In [11]:
asx_codes

,symbol
6674,14D
7639,1AD
7657,1AG
10031,2BE
12587,360
...,...
157914,ZLD
157971,ZMI
157976,ZMM
158005,ZNC


In [12]:
def save_webpage_as_json(url, output_file, folder_name, error_urls):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors
        
        content = response.text  # Get the content of the webpage
        
        # Parse content as JSON
        json_data = json.loads(content)
        
        subfolder_path = os.path.join('stocks', folder_name)
        os.makedirs(subfolder_path, exist_ok=True)
        
        output_path = os.path.join(subfolder_path, output_file)
        
        if os.path.exists(output_path) == False:
            with open(output_path, 'w', encoding='utf-8') as json_file:
                json.dump(json_data, json_file, indent=4)
            print(f"Webpage content saved as {output_file}")
            
    except requests.exceptions.RequestException as e:
        error_urls.append(url)
        print(f"An error occurred: {e}")

In [13]:
# def save_webpage_as_json(url, output_file, folder_name):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Check for any request errors
        
#         content = response.text  # Get the content of the webpage
        
#         # Parse content as JSON
#         json_data = json.loads(content)
        
#         subfolder_path = os.path.join('stocks', folder_name)
#         os.makedirs(subfolder_path, exist_ok=True)
        
#         output_path = os.path.join(subfolder_path, output_file)
#         # Save JSON data as a dictionary
#         with open(output_path, 'w', encoding='utf-8') as json_file:
#             json.dump(json_data, json_file, indent=4)
        
#         print(f"Webpage content saved as {output_file}")
#     except requests.exceptions.RequestException as e:
#         print(f"An error occurred: {e}")

In [14]:
# asx_codes = {"Ticker": ["CSL", "ABX", "A2B", "ASK", "ABC", "ALU", "CAR", "CBA", 'CTD', 'DRR', 'DXS', 'DHG', 'DMP', 'DOW', 'APE', 'FMG', 'GOR', 'GMD']}
# asx_df = pd.DataFrame(asx_codes)

In [15]:
# shz_codes = {"Ticker":["000001", "000002","000004","000005","000006","000007","000008","000009","000010",]}
# shz_df = pd.DataFrame(shz_codes)

In [15]:
def json_scrape(exchange, stock_list, exchange_code_official, error_urls):
    #final_df = pd.DataFrame()
    for i in stock_list['symbol']:
        string_start = "https://eodhistoricaldata.com/api/fundamentals/"
        string_end = "?api_token=64e6bb917d6ca7.00493420"
        url_string = string_start + i + "." + exchange + string_end
       
        output_filename = i + "_" + exchange + ".json"
   
        save_webpage_as_json(url_string, output_filename, exchange_code_official, error_urls)
            

In [16]:
def stock_dataframe(exchange, stock_list, exchange_code_official):
    final_df = pd.DataFrame()
    for i in stock_list['symbol']:
        
        output_filename = i + "_" + exchange + ".json"
        file_path = 'stocks/'+ exchange_code_official +"/"+output_filename
        raw_data = open(file_path)
        dict_data = json.load(raw_data)
   
        new_dict = {'Code':dict_data['General']['Code'], 'Exchange':exchange_code_official, 'Name':dict_data['General']['Name'],'GicSector':dict_data['General']['GicSector'],
                   'GicGroup':dict_data['General']['GicGroup'],'GicIndustry':dict_data['General']['GicIndustry'],'GicSubIndustry':dict_data['General']['GicSubIndustry']}
    
        final_df = final_df.append(new_dict, ignore_index = True)
    return final_df

In [17]:
asx_error = []
#json_scrape("AU", asx_codes, "ASX", asx_error)

An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/A2A.AU?api_token=64e6bb917d6ca7.00493420
An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/AHQDB.AU?api_token=64e6bb917d6ca7.00493420
An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/AYUHC.AU?api_token=64e6bb917d6ca7.00493420
An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/AYUHD.AU?api_token=64e6bb917d6ca7.00493420
An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/BENHB.AU?api_token=64e6bb917d6ca7.00493420
An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/C2FHA.AU?api_token=64e6bb917d6ca7.00493420
An error occurred: 404 Client Error: Not Found for url: https://eodhistoricaldata.com/api/fundamentals/CAMG.AU?api_token=64e6b

In [20]:
asx = stock_dataframe("AU", asx_codes, "ASX")

FileNotFoundError: [Errno 2] No such file or directory: 'stocks/ASX/A2A_AU.json'

In [ ]:
#shz = my_function("SHE", shz_df, "SHZ")

In [ ]:
asx